In [250]:
import pandas as pd
import sqlite3
import networkx as nx
import matplotlib.pyplot as plt
from ipywidgets import interact
import ipywidgets as widgets
import numpy as np
import yfinance as yf
import plotly.express as px
import plotly.graph_objects as go
import statsmodels.api as sm
from statsmodels.tsa.stattools import adfuller

from plotly.offline import download_plotlyjs, init_notebook_mode, iplot
import plotly

In [3]:
con = sqlite3.connect("DVA-data-prod")
q = "select * from tickers"
article_count_complete = pd.read_sql_query(q, con)
con.close()

article_count = article_count_complete.rename(columns={"ticker": "value"})
display(article_count[:10])
article_count.shape

,value,name,article_count
0,A,"Agilent Technologies, Inc",192
1,AA,Alcoa Corporation,117
2,AAALF,Aareal Bank AG,0
3,AAALY,Aareal Bank AG,0
4,AAARF,Aluar Aluminio Argentino S.A.I.C,0
5,AABB,"Asia Broadband, Inc",0
6,AABVF,Aberdeen International Inc,0
7,AACAF,AAC Technologies Holdings Inc,0
8,AACAY,AAC Technologies Holdings Inc,0
9,AACG,ATA Creativity Global,2


(19619, 3)

In [9]:
con = sqlite3.connect("DVA-data-prod")
q = "select * from news_articles"
news = pd.read_sql_query(q, con)
con.close()

news.shape

(29131246, 4)

In [10]:
display(news[:10])

,article_id,ticker1,ticker2,publication_date
0,1997dc291dc36e43fe9f8beb431f6be1,A,GOOGL,2020-04-08T17:54:08.179Z
1,1997dc291dc36e43fe9f8beb431f6be1,A,BRK/A,2020-04-08T17:54:08.179Z
2,1997dc291dc36e43fe9f8beb431f6be1,A,JNJ,2020-04-08T17:54:08.179Z
3,1997dc291dc36e43fe9f8beb431f6be1,A,AMZN,2020-04-08T17:54:08.179Z
4,b6d76307caef5efa6880f13af2cefe4a,A,GOOGL,2020-04-07T23:20:05.135Z
5,b6d76307caef5efa6880f13af2cefe4a,A,LOW,2020-04-07T23:20:05.135Z
6,b6d76307caef5efa6880f13af2cefe4a,A,LBTYA,2020-04-07T23:20:05.135Z
7,b6d76307caef5efa6880f13af2cefe4a,A,HPQ,2020-04-07T23:20:05.135Z
8,b6d76307caef5efa6880f13af2cefe4a,A,BRK/A,2020-04-07T23:20:05.135Z
9,b6d76307caef5efa6880f13af2cefe4a,A,EBAY,2020-04-07T23:20:05.135Z


In [14]:
# Options for dropdown

US_ticks = pd.read_csv("US_LIST_OF_SYMBOLS.csv")
US_ticks = US_ticks[US_ticks['Type']=="Common Stock"]
US_ticks = US_ticks.rename(columns={"Code": "value", "Name": "label"})

us_ticks = US_ticks.to_dict(orient='records')
drop = ['Country', 'Exchange', 'Currency', 'Type']
for i in us_ticks:
    for key in drop:
        i.pop(key)
    
ticks = []
for i, j in enumerate(us_ticks):
    tup = (j['label'], j['value'])
    ticks.append(tup)
ticks[0:3]

[('Agilent Technologies, Inc', 'A'),
 ('Alcoa Corporation', 'AA'),
 ('Aareal Bank AG', 'AAALF')]

In [15]:
############################ Get Company name (label) and Ticker (value) ##########################################
US_ticks = pd.read_csv("US_LIST_OF_SYMBOLS.csv")
US_ticks = US_ticks[US_ticks['Type']=="Common Stock"]
US_ticks = US_ticks.rename(columns={"Code": "value", "Name": "label"})
US_ticks = US_ticks.sort_values(by=['value'])
US_ticks = US_ticks.merge(article_count, on='value', how='left')
US_ticks = US_ticks[US_ticks['article_count'] != 0] # remove tickers with 0 article count

print(US_ticks.shape)


us_ticks = US_ticks.to_dict(orient='records')
drop = ['Country', 'Exchange', 'Currency', 'Type']
for i in us_ticks:
    for key in drop:
        i.pop(key)
    
ticks = []
for i in us_ticks:
    tup = (i['label'], i['value'])
    ticks.append(tup)
ticks.insert(0, ("",""))

(7871, 8)


In [226]:
%%javascript
IPython.OutputArea.prototype._should_scroll = function(lines) {
    return false;
}

<IPython.core.display.Javascript object>

In [492]:
############################# Create Network Graph ############################################
def network_graph(Ticker):
    tic_filter = news[news['ticker1']==Ticker]
    tic_top6 = tic_filter.groupby(['ticker2']).size().reset_index(name='count').sort_values(['count'], ascending=False)
    tic_top6 = tic_top6.iloc[:6]
    tic_list = tic_top6['ticker2'].tolist()
    tic_list.insert(0, Ticker)
    tic_list.insert(0, "")
    tic_top6['ticker1'] = Ticker
    w = tic_top6['count'].tolist()
    weights = [np.sqrt(k)/1.5 for k in w]
    G = nx.from_pandas_edgelist(tic_top6, source='ticker1', target='ticker2')
    nx.draw(G, with_labels=True, node_color='skyblue', node_size=1500, width=weights)
###############     
    def on_button_clicked(ticker_1, ticker_2):
    # Get prices data
        d = {}
        tickers_list = [ticker_1, ticker_2]
        tickers_string = ticker_1+" "+ticker_2
        data = yf.download(tickers_string, start=start_date.value, end=end_date.value, group_by="ticker")
        for ticker in tickers_list:
            d[ticker] = data[ticker]['Adj Close']
        df = pd.DataFrame(data=d)
        
    # Plot prices
        df_plot = pd.DataFrame(data=d)
        df_plot.reset_index(level=0, inplace=True)
        df_plot = pd.melt(df_plot, id_vars=['Date'], value_vars=[ticker_1, ticker_2])
        df_plot = df_plot.rename(columns={"variable": "Ticker", "value": "Price"})        
        fig = px.line(df_plot, x="Date", y="Price", color="Ticker")
        fig.show()
      
    # Engle_Granger test
        X = df.iloc[:,0]
        Y = df.iloc[:,1]
        model = sm.OLS(X, Y)
        model = model.fit()
        df['spread'] = X - model.params[0] * Y

    # Plot spread line
        df_spread = df.copy()
        df_spread.reset_index(level=0, inplace=True)    
        fig = px.line(df_spread, x="Date", y="spread")
        fig.add_hline(y=df['spread'].mean(axis=0), line_dash="dot",
                     annotation_text="spread mean",
                     annotation_position="bottom left")
        fig.add_hline(y=df['spread'].std()*2,
                     annotation_text="+2std",
                     annotation_position="bottom left")
        fig.add_hline(y=df['spread'].std()*-2,
                     annotation_text="-2std",
                     annotation_position="bottom left")
        fig.show()

#     ADF test on spread to test for stationary
        adf = adfuller(df['spread'],regression='nc', maxlag=1)

    # Key values output
        test_statistic = adf[0]
        p_value = adf[1]
        number_of_observations = adf[3]
        critical_values = adf[4]
        confidence_level = (1-p_value)*100
 
        fig = go.Figure(go.Indicator(
        domain = {'x': [0, 1], 'y': [0, 1]},
        value = confidence_level,
        mode = "gauge+number",
        title = {'text': "Cointegration Score"},
        gauge = {'axis': {'range': [None, 100]},
                 'bar': {'color': "darkblue"},
                 'steps' : [
                     {'range': [0, 90], 'color': "red"},
                     {'range': [90, 95], 'color': "yellow"},
                     {'range': [95, 99], 'color': "greenyellow"},
                     {'range': [99, 100], 'color': "green"}],
                 'threshold' : {'line': {'color': "black", 'width': 4}, 'thickness': 0.75, 'value': 100}}))

        fig.show()
######################################################
    
    start_date = widgets.DatePicker(description='Start Date',value=pd.to_datetime('2020-01-02'))
    end_date = widgets.DatePicker(description='End Date', value=pd.to_datetime('2020-11-01'))
#     display(start_date, end_date)
    display(widgets.HBox([start_date, end_date]))
    
    widgets.interact_manual(on_button_clicked, ticker_1 = tic_list, ticker_2 = tic_list)
##############    
###########################################################################
interact(network_graph, Ticker=ticks);

interactive(children=(Dropdown(description='Ticker', options=(('', ''), ('Agilent Technologies, Inc', 'A'), ('…